In [1]:
import numpy as np
import pickle
from pandas import DataFrame, read_csv
from sklearn import preprocessing
import helperModule as hm
import warnings
import tensorflow as tf

warnings.filterwarnings("ignore")

In [2]:
#########################
############ Read in data
#########################

data_train, data_test = hm.readData()
x_train, y_train, x_test, y_test = hm.splitData(data_train, data_test)


In [3]:
#########################
# Mapping of attack types
#########################

'''
38 different attack types are mapped to their corresponding group
-----------
normal                      ==>   NORMAL
unknown                     ==>   UNKNOWN
back,land ..                ==>   DOS
satan,ipsweep,nmap, ..      ==>   PROBE
ftp_write,guess_passwd, ..  ==>   R2L
rootkit,perl ..             ==>   U2R
-----------
'''

y_train, y_test = hm.mapAttackTypes(y_train, y_test)


In [4]:
##########################
# Feature Encoding
##########################

'''
Categorial features (protocol_types, service, flags) are encoded into integers.
-----------
protocol_types:
['tcp' 'udp' 'icmp']                          ==> [0, 1, 2]
service:
['ftp_data', 'telnet', ... 'rje', 'harvest']  ==> [0, 1, .... 67, 68]
flags:
['SF', 'S0', ...  ,'S2', 'OTH']               ==> [ 0, 1 ... , 9, 10]
-----------
'''

x_train = hm.encodeFeatures(x_train)
x_test = hm.encodeFeatures(x_test)


In [5]:
##########################
# Label Encoding
##########################

'''
5 different attack groups are encoded into integers
-----------
NORMAL         ==>   0
DOS            ==>   1
PROBE          ==>   2
R2L            ==>   3
U2R            ==>   4
-----------
'''
y_train = hm.encodeLabels(y_train)
y_test = hm.encodeLabels(y_test)

In [6]:
#########################
####### Binarize Labels
#########################

y_train = hm.binarizeLabels(y_train)
y_test = hm.binarizeLabels(y_test)


In [7]:
#########################
######### Feature Scaling
#########################

'''
Scale non-categorial features into to values between 0 and 1
'''

x_train, x_test = hm.scaleFeatures(x_train, x_test) 


#########################
######### Turn data Frame into matrix 
#########################

x_train = x_train.astype(np.float32).values
x_test = x_test.astype(np.float32).values



In [8]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [9]:
num_nodes= 512
batch_size = 128
num_features = 41
num_labels = 5


In [13]:
graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, num_features))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(x_test)
  #tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([num_features, num_nodes]))
  biases_1 = tf.Variable(tf.zeros([num_nodes]))
  weights_2 = tf.Variable(
    tf.truncated_normal([num_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  relu_layer=tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
  logits = tf.matmul(relu_layer, weights_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.3).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
   tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
  #test_prediction =  tf.nn.softmax(
  #tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)


In [14]:
num_steps = 1000001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (y_train.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = x_train[offset:(offset + batch_size), :]
    batch_labels = y_train[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 5000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), y_test))
  #print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 51.888718
Minibatch accuracy: 4.7%
Validation accuracy: 63.3%
Minibatch loss at step 5000: 0.040026
Minibatch accuracy: 99.2%
Validation accuracy: 75.4%
Minibatch loss at step 10000: 0.048361
Minibatch accuracy: 98.4%
Validation accuracy: 75.9%
Minibatch loss at step 15000: 0.001061
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Minibatch loss at step 20000: 0.002677
Minibatch accuracy: 100.0%
Validation accuracy: 77.1%
Minibatch loss at step 25000: 0.009271
Minibatch accuracy: 99.2%
Validation accuracy: 76.8%
Minibatch loss at step 30000: 0.002279
Minibatch accuracy: 100.0%
Validation accuracy: 77.1%
Minibatch loss at step 35000: 0.007642
Minibatch accuracy: 100.0%
Validation accuracy: 77.0%
Minibatch loss at step 40000: 0.002782
Minibatch accuracy: 100.0%
Validation accuracy: 76.1%
Minibatch loss at step 45000: 0.011810
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at step 50000: 0.004906
Minibatch accuracy: 100.0%
V

KeyboardInterrupt: 